In [1]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np

In [6]:
driver = webdriver.Firefox()
driver.get("https://www.bloomberg.com/citylab")
time.sleep(10)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    try:
        driver.find_element(By.CLASS_NAME, "lazy-load-stories__button").click()
    except Exception as e:
        driver.quit()
        print(e)

# urls = pd.DataFrame(columns=['title', 'url'])
# soup = BeautifulSoup(driver.page_source, "html.parser")
# for i, link in enumerate(soup.select('.listing-thumbnails>div>h3>a')):
#     urls.loc[i] = [link.text, link['href']]

Message: Unable to locate element: .lazy-load-stories__button
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16



MaxRetryError: HTTPConnectionPool(host='localhost', port=56473): Max retries exceeded with url: /session/97e1e242-6ead-4fa7-8eec-61112f39ba04/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12d552280>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [79]:
urls

,title,url
0,Economics in Brief: Starbucks Union Wave Conti...,https://nextcity.org/urbanist-news/economics-i...
1,Displacement-Fighting Group in L.A.’s Little T...,https://nextcity.org/urbanist-news/displacemen...
2,Housing Briefs: New York State’s Budget Fails ...,https://nextcity.org/urbanist-news/housing-bri...
3,How a Tennessee Housing Policy Concentrates Po...,https://nextcity.org/urbanist-news/how-a-tenne...
4,Too Hot? Your City Needs a New Coat of Paint,/podcast/too-hot-your-city-needs-a-new-coat-of...
...,...,...
1179,"Bringing the Arts to Prison, Even During a Pan...",https://nextcity.org/urbanist-news/bringing-th...
1180,How A Historic Arts Institution Can Spur Neigh...,https://nextcity.org/webinars/how-a-historic-a...
1181,How To Reduce The Voting Age to 16,https://nextcity.org/webinars/how-to-reduce-th...
1182,Stealth Food Banks Serve the Undocumented,https://nextcity.org/urbanist-news/stealth-foo...


In [80]:
urls.to_csv('next-city-links.csv')

In [3]:
urls = pd.read_csv('next-city-links.csv', index_col=0)

In [84]:
corpus = pd.DataFrame()

In [85]:
for u in tqdm(urls.iterrows(), ncols=len(urls)):
    url = u[1].url
    if 'podcast' in url:
        continue
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        date = soup.select('p.daily-byline')[0].text.split('\xa0')[1].strip()
    except IndexError as e:
        date = pd.NA
    text = ""
    for p in soup.select('div.entry-content>p'):
        text = text + p.text
    corpus.loc[u[0], ["title", "url", "date", "text"]] = [u[1].title, u[1].url, date, text]    

1184it [22:03,  1.12s/it]


In [86]:
corpus

,title,url,date,text
0,Economics in Brief: Starbucks Union Wave Conti...,https://nextcity.org/urbanist-news/economics-i...,"April 15, 2022",Starbucks Union Wave Continues Workers at a St...
1,Displacement-Fighting Group in L.A.’s Little T...,https://nextcity.org/urbanist-news/displacemen...,"April 14, 2022",Displacement runs through the tapestry of Amer...
2,Housing Briefs: New York State’s Budget Fails ...,https://nextcity.org/urbanist-news/housing-bri...,"April 14, 2022",New York State’s Budget Fails to Provide Vouch...
3,How a Tennessee Housing Policy Concentrates Po...,https://nextcity.org/urbanist-news/how-a-tenne...,"April 14, 2022",Latoya Akinyemi watches her youngest children ...
5,What We Learn From Black- and Women-Led Cooper...,https://nextcity.org/urbanist-news/what-we-lea...,"April 13, 2022",EDITOR’S NOTE: The following is an excerpt fro...
...,...,...,...,...
1179,"Bringing the Arts to Prison, Even During a Pan...",https://nextcity.org/urbanist-news/bringing-th...,"August 5, 2020","This March, the nonprofit Rehabilitation Throu..."
1180,How A Historic Arts Institution Can Spur Neigh...,https://nextcity.org/webinars/how-a-historic-a...,<NA>,
1181,How To Reduce The Voting Age to 16,https://nextcity.org/webinars/how-to-reduce-th...,<NA>,
1182,Stealth Food Banks Serve the Undocumented,https://nextcity.org/urbanist-news/stealth-foo...,"August 4, 2020","Underground food distributions, direct financi..."


In [87]:
corpus.to_csv('next-city-corpus.csv')